![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# YOLOv12 Finetuning

## Aprendizaje Automático Aplicado

### Maestría en Ciencia de Datos

#### **Julio Waissman**, 2025

[**Abrir en google Colab**](https://colab.research.google.com/github/mcd-unison/aaa-curso/blob/main/ejemplos/YOLOv12_fine_tuning.ipynb)


## YOLO (You Only Look Once)

El modelo YOLO (You Only Look Once) para la detección de objetos funciona de una manera bastante ingeniosa. En lugar de analizar múltiples regiones de una imagen como otros detectores, YOLO divide la imagen en una cuadrícula. Cada celda de esta cuadrícula es responsable de predecir objetos cuyos centros caen dentro de ella. Para cada celda, el modelo predice simultáneamente múltiples cajas delimitadoras (rectángulos que encierran los objetos), la confianza de que cada caja contenga un objeto, y las probabilidades de clase de ese objeto (por ejemplo, perro, coche, persona).

Esencialmente, YOLO realiza una única pasada a través de la red neuronal para hacer todas estas predicciones a la vez, lo que le confiere su gran velocidad. Las cajas delimitadoras predichas se filtran según su puntuación de confianza y se aplican técnicas como la supresión no máxima para eliminar las detecciones redundantes del mismo objeto. Este enfoque de "mirar una sola vez" permite una detección de objetos en tiempo real muy eficiente.

El artículo original que introduce la arquitectura YOLO es [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640). De la versión original se han derivado varios modelos con mejoras, siendo el último (a la fecha de este ejemplo) la versión 12, de febrero de 2025.

YOLO12 introduce una arquitectura centrada en la atención que se aleja de los enfoques tradicionales basados en CNN utilizados en modelos YOLO anteriores, pero conserva la velocidad de inferencia en tiempo real esencial para muchas aplicaciones. Este modelo consigue la máxima precisión en la detección de objetos gracias a novedosas innovaciones metodológicas en los mecanismos de atención y en la arquitectura general de la red, al tiempo que mantiene el rendimiento en tiempo real. El reporte técnico donde se presenta es [YOLOv12: Attention-Centric Real-Time Object Detectors](https://arxiv.org/pdf/2502.12524).

En general los modelos YOLO son abiertos y libres de uso con licencias tipo [GNU Affero General Public License v3.0](https://github.com/sunsmarterjie/yolov12/blob/main/LICENSE). Los modelos se distributen a través de la empresa [Ultralytics](https://www.ultralytics.com/es/yolo) y para realizar el *fine tuning* lo más complicado es poner el conjunto de datos de interés en el formato `COCODataset`.

Veamos como hacer esto.

## 1. Instalando y cargando modulos

In [ ]:
# Vamos ainstalar algunas liberias que no vienen en colab por default

# Para usar los modelos YOLO
!pip install ultralytics --quiet

# Para recuperar los datos que vamos a usar de ejemplo
!pip install kagglehub --quiet



In [ ]:
import os
from ultralytics import YOLO
import kagglehub
import cv2
from tqdm import tqdm
from PIL import Image
import xml.etree.ElementTree as ET
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline


## 2. Descargando datos de Kaggle

Para este ejemplo vamos a usar un conjunto de datos que se usa comunmente para detección de objetos, el [HRSC2016-MS Dataset](https://www.kaggle.com/datasets/weiming97/hrsc2016-ms-dataset) que son datos a multiples resoluciones donde se detectan navíos en imagenes de Google Earth.

In [ ]:
dataset_path = "/content/HRSC2016-MS/"

path = kagglehub.dataset_download("weiming97/hrsc2016-ms-dataset")
print("Los archivos se descargaron en:", path)
print("Pero los vamos a pasar a: ", dataset_path)

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
    os.system(f"cp -r {path}/* {dataset_path}")

## 3. Creando un directorio con datos al estilo COCODataset

La parte complicada es como leer el archivo en formato XML con la información de los objetos, tal como está codificado en al base de datos y convertirlos al formato estandar que utiliza YOLO

In [ ]:
def xml2txt(xml_file_path, txt_file, w, h):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    objs = root.findall('object')

    if not objs:
        print(f"Warning: No objects found in {xml_file_path}. Creating an empty annotation file.")
        with open(txt_file, 'w') as f:
            f.write("\n")  # Empty file for compatibility
        return

    with open(txt_file, 'w') as f:
        for obj in objs:
            try:
                xmin = int(obj.find('bndbox/xmin').text)
                ymin = int(obj.find('bndbox/ymin').text)
                xmax = int(obj.find('bndbox/xmax').text)
                ymax = int(obj.find('bndbox/ymax').text)

                # Convert to YOLO format (normalized values)
                x_center = (xmin + xmax) / (2.0 * w)
                y_center = (ymin + ymax) / (2.0 * h)
                box_w = (xmax - xmin) / w
                box_h = (ymax - ymin) / h

                # Save in YOLO format
                f.write(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")
            except Exception as e:
                print(f"Error processing {xml_file_path}: {e}")


Ahora creamos un sistema de archivos como el que necesitamos y lo guardamas en `yolo_dir`:

In [ ]:
cwd = os.getcwd()
yolo_dir = os.path.join(cwd, 'HRSC-YOLO')
os.mkdir(yolo_dir)

for i in ['train', 'val', 'test']:
    folder = os.path.join(yolo_dir, i)
    os.mkdir(folder)
    for j in ['images', 'labels']:
        subfolder = os.path.join(folder, j)
        os.mkdir(subfolder)

y ahora leemos los archivos en el formato que vienen en el ejemplo y los guardamos como se solicita en Yolo, vamos a hacer una función para generalizar este proceso:

In [ ]:
def convierte_imagenes_para_yolo(tipo, dataset_path, yolo_dir):
    if tipo not in ['train', 'val', 'test']:
        raise ValueError("tipo debe ser 'train', 'val' or 'test'")
    
    with open(os.path.join(dataset_path, f'ImageSets/{tipo}.txt'), 'r') as f:
        lista_tipo = f.read().splitlines()
    
    for file_name in tqdm(lista_tipo): 
        bmp_path = os.path.join(dataset_path, f'AllImages/{file_name}.bmp') 
        xml_path = os.path.join(dataset_path, f'Annotations/{file_name}.xml') 
        txt_path = os.path.join(yolo_dir, f'{tipo}/labels/{file_name}.txt') 
        png_path = os.path.join(yolo_dir, f'{tipo}/images/{file_name}.png')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)
        img.save(png_path, format='PNG')
    

Y ahora generamos las imágenes al estilo YOLO:

In [ ]:
convierte_imagenes_para_yolo('train', "/content/HRSC2016-MS", yolo_dir)
convierte_imagenes_para_yolo('val', "/content/HRSC2016-MS", yolo_dir)
convierte_imagenes_para_yolo('test', "/content/HRSC2016-MS", yolo_dir)

## 4. Volver a acomodar las imágenes y crear el archivo yaml

De nuevo vamos a poner todas las imágenes juntas y todas las etiquetas juntas, sin que nos importe cuales son de entrenamiento, prueba o validación. Y vamos a rehacer conjuntos nuevos de entrenamiento, prueba y validación a partir de todas las imagenes juntas.

Empezamos por ponerlas todas juntas:

In [ ]:
source_dir = "/content/HRSC-YOLO/"
dest_dir = "/content/experimentals/"

os.makedirs('experimentals', exist_ok=True)
os.makedirs(os.path.join(dest_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "labels"), exist_ok=True)

for split in ['train', 'val', 'test']:
    image_source = os.path.join(source_dir, split, "images")
    label_source = os.path.join(source_dir, split, "labels")

    for file in os.listdir(image_source):
        src_path = os.path.join(image_source, file)
        dest_path = os.path.join(dest_dir, "images", file)
        shutil.copy2(src_path, dest_path)

    for file in os.listdir(label_source):
        src_path = os.path.join(label_source, file)
        dest_path = os.path.join(dest_dir, "labels", file)
        shutil.copy2(src_path, dest_path)

Y aqui, de manera bastante complicada, reconstruyen un nuevo conjunto de entrenamiento, prueba y validación, usando 610 imagenes de entrenamiento, 460 de validación y 610 de prueba:

In [ ]:
# Define paths
base_dir = "/content/experimentals"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

# Output directories
output_dirs = {
    "train": "train",
    "val": "val",
    "test": "test"
}

# Create train, val, and test directories with images and labels subfolders
for split in output_dirs.values():
    os.makedirs(os.path.join(split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split, "labels"), exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(images_dir) if f.endswith(".png")]
random.shuffle(image_files)  # Shuffle dataset

# Define split sizes
train_size, val_size, test_size = 610, 460, 610

# Split dataset
train_images = image_files[:train_size]
val_images = image_files[train_size:train_size + val_size]
test_images = image_files[train_size + val_size:train_size + val_size + test_size]

# Function to copy files
def copy_files(image_list, split):
    for img_name in image_list:
        label_name = img_name.replace(".png", ".txt")  # Corresponding label
        shutil.copy(os.path.join(images_dir, img_name), os.path.join(split, "images", img_name))
        shutil.copy(os.path.join(labels_dir, label_name), os.path.join(split, "labels", label_name))

# Copy files to respective folders
copy_files(train_images, "train")
copy_files(val_images, "val")
copy_files(test_images, "test")


Y ahora establecemos el archivo YAML para el entrenamiento, que podría haber sido con el conjunto tal como lo tenñiamos originalmente.

In [ ]:
yolo_dir2 = "/content"
data_yaml = os.path.join(yolo_dir2, "data.yaml")

yaml_content = f"""
train: {os.path.join(yolo_dir2, 'train/images')}
val: {os.path.join(yolo_dir2, 'val/images')}
test: {os.path.join(yolo_dir2, 'test/images')}

nc: 1
names: ['ship']
"""

with open(data_yaml, "w") as f:
    f.write(yaml_content)

## 5. Cargando el modelo y entrenandolo

Esto es la parte menos divertida con todo lo que ya viene en la librería de Ultralitics.

In [ ]:
# Descarga YOLOv12-large model de https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12l.pt and se sube a colab

!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12l.pt -O \
    /content/yolo12l.pt

Y ahora solo carga y entrena el modelo con las herramientas que ya existen:

In [ ]:
model = YOLO("/content/yolo12l.pt")

model.train(
    data=data_yaml, 
    epochs=100, 
    batch=8, 
    imgsz=640, 
    device='cuda', 
    workers=4, 
    save=True, 
    save_period=10
)
model.val()
print(model)


Y la validación:

In [ ]:
mejor = "/content/runs/detect/train/weights/best.pt"
modelo = YOLO(mejor)
metrics = model.val(data=data_yaml)
print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)

### 6. Haciendo inferencias

In [ ]:
def infer_yolov12(image_path, model_weights):
    """
    path to the saved model weights 
    can be found at runs/detect/trainx/weights/best.pt
    
    """
    model = YOLO(model_weights)
    results = model(image_path)
    
    for result in results:
        img = cv2.imread(image_path)
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0])
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [ ]:
test_image = "/content/HRSC-YOLO/train/images/100001058.png"
infer_yolov12(test_image)